In [5]:
from PyAnvilEditor.pyanvil import World, BlockState
opened_world = World('dataset/minecraft_worlds/Drehmal v2.1.1 PRIMORDIAL minified', save_location=None, write=False, debug=False)

In [6]:
opened_world

In [7]:
opened_world.get_chunk((0,0))

In [8]:
opened_world.chunks

{(0, 0): <PyAnvilEditor.pyanvil.world.Chunk at 0x7ff348df5fa0>}

In [9]:
str(opened_world.get_block((4,60,421)))

'Block(BlockState(minecraft:stone, {}), 0, 0)'

In [1]:
import PyAnvilEditor.pyanvil.nbt as nbt

{1: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 2: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 3: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 4: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 5: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 6: <class 'PyAnvilEditor.pyanvil.nbt.create_simple_nbt_class.<locals>.DataNBTTag'>, 7: <class 'PyAnvilEditor.pyanvil.nbt.create_array_nbt_class.<locals>.ArrayNBTTag'>, 8: <class 'PyAnvilEditor.pyanvil.nbt.create_string_nbt_class.<locals>.DataNBTTag'>, 9: <class 'PyAnvilEditor.pyanvil.nbt.create_list_nbt_class.<locals>.ListNBTTag'>, 10: <class 'PyAnvilEditor.pyanvil.nbt.create_compund_nbt_class.<locals>.CompundNBTTag'>, 11: <class 'PyAnvilEditor.pyanvil.nbt.create_array_nbt_class.<locals>.ArrayNBTTag'>, 12: <class 'PyAnvilEditor.pyanvil.nbt.create_array_nbt_class.<locals>.A

In [2]:
import PyAnvilEditor.pyanvil.stream as stream, gzip
with gzip.open('dataset/minecraft_worlds/Drehmal v2.1.1 PRIMORDIAL/level.dat', mode='rb') as level:
    # Read the file in to an input stream
    in_stream = stream.InputStream(level.read())
    # Decode the stream
    level_data = nbt.parse_nbt(in_stream)
    # Get the value of Data > LevelName
    lvl_name = level_data.get("Data").get("LevelName").get()
    # lvl_name is now a string containing the name of the level used
    
level_data

CompundTag:  size 1 = {CompundTag: Data size 42 = {IntTag 'WanderingTraderSpawnChance' = 75, DoubleTag 'BorderCenterZ' = 0.0, ByteTag 'Difficulty' = 2, LongTag 'BorderSizeLerpTime' = 0, ByteTag 'raining' = 0, LongTag 'Time' = 176533919, IntTag 'GameType' = 2, ListTag: ServerBrands size 5 = [StringTag: None = 'vanilla', StringTag: None = 'fabric', StringTag: None = 'forge', StringTag: None = 'Spigot', StringTag: None = 'Paper'], DoubleTag 'BorderCenterX' = 0.0, DoubleTag 'BorderDamagePerBlock' = 0.20000000298023224, DoubleTag 'BorderWarningBlocks' = 5.0, CompundTag: WorldGenSettings size 4 = {ByteTag 'bonus_chest' = 0, LongTag 'seed' = 27594263, ByteTag 'generate_features' = 1, CompundTag: dimensions size 3 = {CompundTag: minecraft:overworld size 2 = {CompundTag: generator size 4 = {StringTag: settings = 'minecraft:overworld', LongTag 'seed' = 27594263, CompundTag: biome_source size 3 = {LongTag 'seed' = 27594263, ByteTag 'large_biomes' = 0, StringTag: type = 'minecraft:vanilla_layered'

In [3]:
type(level_data)

PyAnvilEditor.pyanvil.nbt.create_compund_nbt_class.<locals>.CompundNBTTag

In [4]:
level_data.get('Data').get('WanderingTraderSpawnChance')

IntTag 'WanderingTraderSpawnChance' = 75

In [5]:
import pickle

In [6]:
with open("test_level_data.pkl", "wb") as f:
    pickle.dump(level_data, f)

AttributeError: Can't pickle local object 'create_compund_nbt_class.<locals>.CompundNBTTag'

---

In [30]:
created_compoundtag = nbt.create_compund_nbt_class(10)

In [31]:
created_compoundtag

PyAnvilEditor.pyanvil.nbt.create_compund_nbt_class.<locals>.CompundNBTTag

In [32]:
with open("test_level_data.pkl", "wb") as f:
    pickle.dump(created_compoundtag, f)

AttributeError: Can't pickle local object 'create_compund_nbt_class.<locals>.CompundNBTTag'

In [58]:
parsers2 = {}

class CompundNBTTag2:

    clazz_id = 10

    @classmethod
    def parse(cls, stream, name):
        tag = cls(tag_name=name)
        while stream.peek() != 0: # end tag
            tag.add_child(parse_nbt(stream))
        stream.read(1) # get rid of the end tag
        return tag

    def __init__(self, tag_name='None', children=[]):
        self.tag_name = tag_name
        self.children = { c.tag_name: c for c in children[:] }
    
    def add_child(self, tag):
        self.children[tag.tag_name] = tag

    def get(self, name):
        return self.children[name]

    # def get(self):
    #     return { n: v.get() for n, v in self.children }

    def name(self):
        return self.tag_name

    def has(self, name):
        return name in self.children

    def to_dict(self):
        nd = {}
        for p in self.children:
            nd[p] = self.children[p].get()
        return nd

    def print(self, indent=''):
        print(indent + 'Compound: ' + self.tag_name + ' size ' + str(len(self.children)))
        for c in self.children:
            self.children[c].print(indent + '  ')

    def serialize(self, stream, include_name=True):
        if include_name:
            stream.write(type(self).clazz_id.to_bytes(1, byteorder='big', signed=False))
            write_string(stream, self.tag_name)
        
        for tag_name in self.children:
            self.children[tag_name].serialize(stream, include_name=True)
        
        stream.write((0).to_bytes(1, byteorder='big', signed=False))

    def clone(self):
        return type(self)(tag_name=self.tag_name, children=[v.clone() for k, v in self.children.items()])

    def __repr__(self):
        str_dat = ', '.join([c.__repr__() for name, c in self.children.items()])
        return f'CompundTag: {self.tag_name} size {str(len(self.children))} = {{{str_dat}}}]'

    def __eq__(self, other):
        passed = True
        for name, v in self.children.items():
            if name not in other.children:
                passed = False
            elif other.children[name] != v:
                passed = False
        return self.tag_name == other.tag_name and \
            len(self.children) == len(other.children) and \
            passed
    
parsers2[10] = CompundNBTTag2          
    
# return CompundNBTTag2

In [59]:
created_compoundtag2 = CompundNBTTag2()
parsers2

{10: __main__.CompundNBTTag2}

In [60]:
with open("test_level_data2.pkl", "wb") as f:
    pickle.dump(created_compoundtag2, f)

---

In [70]:
def foo(bar, baz) :
    def closure(waldo) :
        return baz * waldo
    return closure

In [71]:
foo(2, 3)(4)

12

In [72]:
class foo(object):
    def __init__(self, bar, baz):
        self.baz = baz
    def __call__(self,waldo):
        return self.baz * waldo

In [73]:
foo(2, 3)(4)

12